In [8]:
import sys
import os
import json
from datetime import datetime, date
from pathlib import Path

# Add BOOST Python reference implementation to path
boost_python_path = Path("../../../../drafts/current/reference-implementations/python")
schema_path = Path("../../../../drafts/current/schema")

# Verify paths exist
if not boost_python_path.resolve().exists():
    raise FileNotFoundError(f"Python implementation not found at {boost_python_path.resolve()}")
if not schema_path.resolve().exists():
    raise FileNotFoundError(f"Schema directory not found at {schema_path.resolve()}")

sys.path.insert(0, str(boost_python_path.resolve()))

# Import BOOST client
from boost_client import create_client

print("✅ BOOST Python reference implementation imported successfully")
print(f"✅ Schema directory found at {schema_path.resolve()}")


✅ BOOST Python reference implementation imported successfully
✅ Schema directory found at /Users/lkilroy/Documents/BOOST/drafts/current/schema


In [9]:
# Initialize BOOST client with schema path
schema_path = str(Path("../../../../drafts/current/schema").resolve())
client = create_client(schema_path=schema_path)

# Get schema information
schema_info = client.get_schema_info()
print(f"BOOST client initialized with v2.5.1 features")
print(f"Available entities: {schema_info['available_entities']}")
print(f"Schema loader: {schema_info['schema_loader']}")
print(f"Validator: {schema_info['validator']}")


TypeError: 'type' object is not iterable

In [ ]:
# Create Sherwood Station organization
sherwood = client.create_organization(
    organization_id="ORG-SHERWOOD-001",
    organizationName="Sherwood Station Power Company",
    organizationType="producer",
    contactEmail="contact@sherwoodstation.com",
    contactPhone="+1-555-123-4567",
    regulatedEntityType="producer",
    operationalStatus="active",
    facilityCapacity={
        "annualCapacity": 500000,
        "unit": "MJ"
    }
)

# Validate organization
validation = client.validate_entity(sherwood)
print("Organization Validation:")
if validation['valid']:
    print("Organization validated successfully")
else:
    print("Validation errors:", validation['schema_errors'])


In [ ]:
# Create organization with None fields
org_with_none = client.create_organization(
    organization_id="ORG-TEST-001",
    organizationName="Test Organization",
    organizationType="supplier",
    contactEmail="contact@test.com",
    contactPhone=None,  # Optional field
    regulatedEntityType="supplier",
    operationalStatus="active",
    facilityCapacity=None  # Optional field
)

# Validate organization with None fields
validation = client.validate_entity(org_with_none)
print("None Field Handling Test:")
if validation['valid']:
    print("Organization with None fields validated successfully")
else:
    print("Validation errors:", validation['schema_errors'])


In [ ]:
# Create a transaction
transaction = client.create_transaction(
    transaction_id="TXN-2025-Q1-001",
    organization_id="ORG-SHERWOOD-001",
    transaction_date=datetime.now().isoformat(),
    contract_value=75000.00,
    contract_currency="USD",
    transaction_status="completed",
    feedstock_type="mill_waste",
    feedstock_volume=500,
    feedstock_unit="bone_dry_tonnes",
    pathway_id="PATH-BIORAM-001"
)

# Validate transaction
validation = client.validate_entity(transaction)
print("Transaction Validation:")
if validation['valid']:
    print("Transaction validated successfully")
else:
    print("Validation errors:", validation['schema_errors'])


In [ ]:
# Export all entities to JSON-LD
jsonld_output = client.export_to_jsonld()

print("JSON-LD Export:")
print(json.dumps(json.loads(jsonld_output), indent=2))


In [ ]:
# Perform comprehensive validation
validation_results = client.validate_all()

print("Comprehensive Validation Results:")
if validation_results['valid']:
    print("All entities passed comprehensive validation")
else:
    print("Validation errors found:")
    for error in validation_results['errors']:
        print(f"- {error}")


# Sherwood Station Power Company BIORAM Example

This notebook demonstrates the BIORAM compliance example for an imagined company using the BOOST Python reference implementation with proper schema validation and JSON-LD compatibility. 

## Key Features

- **Schema Compliance**: All entities conform to BOOST core schemas
- **Dynamic Validation**: Uses schema-driven validation from the Python reference implementation
- **JSON-LD Support**: Proper semantic web compatibility with `@context`, `@type`, `@id`
- **Automatic Enum Validation**: Uses current schema enums for field validation
- **ID Pattern Compliance**: Follows BOOST ID conventions (`ORG-*`, `TXN-*`, etc.)
- **Future-Proof Design**: Adapts automatically to schema changes

## Set up and import BOOST Client


In [ ]:
import sys
import os
import json
from datetime import datetime, date
from pathlib import Path

# Add BOOST Python reference implementation to path
boost_python_path = Path("../../../../drafts/current/reference-implementations/python")
sys.path.insert(0, str(boost_python_path.resolve()))

# Import BOOST client
from boost_client import create_client

print("✓ BOOST Python reference implementation imported successfully")

In [ ]:
# Initialize BOOST client with schema path
schema_path = str(Path("../../../../drafts/current/schema").resolve())
client = create_client(schema_path=schema_path)

# Get schema information
schema_info = client.get_schema_info()
print(f"✓ BOOST client initialized")
print(f"Available entities: {schema_info['available_entities']}")
print(f"Schema loader: {schema_info['schema_loader']}")
print(f"Validator: {schema_info['validator']}")

## Dynamic Schema Introspection

Let's explore what enum values are available from the current schemas. 


In [ ]:
# Get schema information to verify available entities
schema_info = client.get_schema_info()
print("Available entities:", schema_info['available_entities'])

# Get available enum values from schemas
try:
    org_types = client.get_available_enum_values('organization', 'organizationType')
    print("\nOrganization types:", org_types)
except Exception as e:
    print("\nCould not get organization types:", str(e))

try:
    transaction_statuses = client.get_available_enum_values('transaction', 'transactionStatus')
    print("\nTransaction statuses:", transaction_statuses)
except Exception as e:
    print("\nCould not get transaction statuses:", str(e))

## Create BOOST-compliant Organization 

Let's create the Sherwood Station Power Company organization using proper BOOST schema validation. 

In [ ]:
org_id = client.generate_id("organization", "SHERWOOD-STATION")
print(f"Generated organization ID: {org_id}")

sherwood_station = client.create_organization(
    organization_id=org_id,
    organizationName="Sherwood Station Power Company",
    organizationType="producer",
    contactEmail="contact@sherwoodstation.com",
    contactPhone="+1-555-123-4567",
    regulatedEntityType="producer",
    operationalStatus="active",
    facilityCapacity={
        "annualCapacity": 500000,
        "unit": "MJ"
    }
)


## Create Upstream Organizations

Since Sherwood is an end consumer of mill waste, let's create the upstream suppliers that provide Sherwood with feedstock. 


In [ ]:
suppliers = []
supplier_data = [
    {
        "name": "Anderson Valley Lumber",
        "suffix": "AVL-001"
    },
    {
        "name": "Sierra Forest Products",
        "suffix": "SFP-001"
    }
]

for supplier_info in supplier_data:
    supplier_id = client.generate_id("organization", supplier_info["suffix"])
    supplier = client.create_organization(
        organization_id=supplier_id,
        organizationName=supplier_info["name"],
        organizationType="supplier",
        contactEmail=f"contact@{supplier_info['suffix'].lower()}.com",
        contactPhone="+1-555-123-4567",
        regulatedEntityType="supplier",
        operationalStatus="active"
    )
    suppliers.append(supplier)
    
    # Validate supplier
    validation = client.validate_entity(supplier)
    if validation['valid']:
        print(f"✅ Created and validated supplier: {supplier_info['name']} (ID: {supplier_id})")
    else:
        print(f"❌ Validation errors for {supplier_id}:", validation['schema_errors'])



## Create BOOST-compliant Transactions

Now, let's create transactions using the proper BOOST transaction schema with all required fields. 



In [ ]:
# Create one transaction per supplier
transactions = []

for i, supplier in enumerate(suppliers):
    transaction = client.create_transaction(
        transaction_id=client.generate_id("transaction", f"2025-Q1-{i+1:03d}"),
        organization_id=supplier.organizationId,
        customer_id=sherwood.organizationId,
        transaction_date="2025-01-15",
        contract_value=75000.00,
        contract_currency="USD",
        transaction_status="completed",
        feedstock_type="mill_waste",
        feedstock_volume=500,
        feedstock_unit="bone_dry_tonnes"
    )
    transactions.append(transaction)
    
    # Validate transaction
    validation = client.validate_entity(transaction)
    if validation['valid']:
        print(f"Transaction {transaction.transactionId} validated successfully")
    else:
        print(f"Validation errors for {transaction.transactionId}:", validation['schema_errors'])

print(f"\nCreated {len(transactions)} transactions")

In [ ]:
# create BOOST compliant transactions 
boost_transactions = []

for i, txn_data in enumerate(original_transactions, 1):
    # Generate proper BOOST transaction ID
    txn_id = client.generate_id("transaction", f"2025-Q1-{i:03d}")
    
    # Get customer organization data
    supplier = suppliers[txn_data["supplierIndex"]]
    supplier_data = supplier.model_dump(by_alias=True)
    
    # Generate proper Supplier ID referencing the organization
    supplier_id = client.generate_id("transaction", f"SUPP-{supplier_data['organizationId'].replace('ORG-', '')}")
    
    # Calculate contract value
    contract_value = txn_data["feedstockVolume"] * txn_data["feedstockPrice"]
    transaction = client.create_transaction(
    # Standard BOOST fields
        transaction_id=txn_id,
        organization_id=supplier_data["organizationId"],
        customer_id=supplier_data["organizationId"],
        transaction_date=txn_data["transactionDate"],
        contract_value=contract_value,
        contract_currency="USD",
        transaction_status="completed",
        feedstock_type=txn_data["feedstockType"],
    
        # BIORAM specific fields - aligned with pathways
        pathwayId=txn_data["pathwayId"],  # Link to BIORAM pathway
        feedstockCategory=txn_data["feedstockCategory"],  # Match pathway terminology
        volumeDelivered=txn_data["feedstockVolume"],
        volumeUnit="bone-dry_tonnes",  # Consistent with pathway limits
        energyProduct="electricity",  # Match pathway
    
        # Additional BIORAM tracking fields
        haulDistance=txn_data["haulDistance"],
        haulUnit="mile",
        fireHazardZoneStatus=txn_data["fireHazardSensitivityZoneStatus"],
        reportingPeriod="2025-Q1"
    )
    boost_transactions.append(transaction)
    print(f"Created transaction: {txn_id}")

print(f"Created {len(boost_transactions)} transactions")
        

        